<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#START-Classification-Models---Data-Set-Variations" data-toc-modified-id="START-Classification-Models---Data-Set-Variations-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>START Classification Models - Data Set Variations</a></span></li><li><span><a href="#Import-Data-Sets" data-toc-modified-id="Import-Data-Sets-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import Data Sets</a></span><ul class="toc-item"><li><span><a href="#Data-Set-I." data-toc-modified-id="Data-Set-I.-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Data Set I.</a></span></li></ul></li><li><span><a href="#import-btc-data" data-toc-modified-id="import-btc-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>import btc data</a></span></li><li><span><a href="#calculate-meier-multiple:-200-moving-average-current-price" data-toc-modified-id="calculate-meier-multiple:-200-moving-average-current-price-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>calculate meier multiple: 200 moving average current price</a></span></li><li><span><a href="#transfers-to/from-exchanges-diff-und-sliding-window" data-toc-modified-id="transfers-to/from-exchanges-diff-und-sliding-window-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>transfers to/from exchanges diff und sliding window</a></span></li><li><span><a href="#kick-out-meyer-multiple-eth-because-of-data-loss" data-toc-modified-id="kick-out-meyer-multiple-eth-because-of-data-loss-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>kick out meyer multiple eth because of data loss</a></span></li><li><span><a href="#Bring-df_dataset1-and-df_dataset_btc-together" data-toc-modified-id="Bring-df_dataset1-and-df_dataset_btc-together-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Bring df_dataset1 and df_dataset_btc together</a></span><ul class="toc-item"><li><span><a href="#Prediction-Shift" data-toc-modified-id="Prediction-Shift-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Prediction Shift</a></span><ul class="toc-item"><li><span><a href="#Data-Set-I." data-toc-modified-id="Data-Set-I.-7.1.1"><span class="toc-item-num">7.1.1&nbsp;&nbsp;</span>Data Set I.</a></span></li></ul></li></ul></li><li><span><a href="#get-yearly-btc-dollar-increase-for-traditional-investment" data-toc-modified-id="get-yearly-btc-dollar-increase-for-traditional-investment-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>get yearly btc dollar increase for traditional investment</a></span></li><li><span><a href="#Classification-Setup" data-toc-modified-id="Classification-Setup-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Classification Setup</a></span><ul class="toc-item"><li><span><a href="#Create-Data-Set-Copies-for-Trading-Strategy" data-toc-modified-id="Create-Data-Set-Copies-for-Trading-Strategy-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>Create Data Set Copies for Trading Strategy</a></span></li><li><span><a href="#Categorization---Data-Set-I." data-toc-modified-id="Categorization---Data-Set-I.-9.2"><span class="toc-item-num">9.2&nbsp;&nbsp;</span>Categorization - Data Set I.</a></span></li></ul></li><li><span><a href="#Model-Preparation---Data-Set-I." data-toc-modified-id="Model-Preparation---Data-Set-I.-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Model Preparation - Data Set I.</a></span><ul class="toc-item"><li><span><a href="#Train-Test-Split---Data-Set-I." data-toc-modified-id="Train-Test-Split---Data-Set-I.-10.1"><span class="toc-item-num">10.1&nbsp;&nbsp;</span>Train Test Split - Data Set I.</a></span></li></ul></li><li><span><a href="#predict-on-tomorrow" data-toc-modified-id="predict-on-tomorrow-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>predict on tomorrow</a></span></li><li><span><a href="#get-x-train-data-of-today-to-predict-tomorrow-y" data-toc-modified-id="get-x-train-data-of-today-to-predict-tomorrow-y-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>get x train data of today to predict tomorrow y</a></span></li></ul></div>

In [1]:
#1.1  Data Import via Application Programming Interfaces

In [2]:
# Import of Modules and Packages
import os
import requests
import json
import datetime
import yfinance as yf
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# Adjustment of Decimal Places
pd.options.display.float_format = '{:.2f}'.format

In [4]:
setdir='C:\\Users\\markus.gebele\\github_repo\\JupyterLabDir\\Rest\\MA BTC\\Markus_Code_MA_final\\20201010'
os.chdir(setdir)

In [5]:
# Glassnode API Key
API_Key =''

In [6]:
#3  Btc Eth

In [21]:
def import_glassnode(url, feature_name):
    data = requests.get(url).json()
    df = pd.json_normalize(data)

    # Rename columns
    df.columns = ['Date', feature_name]

    # Convert Object to Datetime Object
    df.Date = pd.to_datetime(df.Date, unit='s')
    #df.Date = df.Date.map(lambda x: x.strftime('%Y-%m-%d'))

    # Set Date Column as Index
    df.set_index('Date', inplace=True)

    # Drop Missing Values
    df = df.dropna()

    # Change Data Type to Float
    df = df.astype(float)

    # Creation of CSV-File, Part of Feature Name as CSV File Name
    feature_name = feature_name.replace('/', '_')
    df.to_csv("{}.csv".format(feature_name))


# Plot of DataFrame
    return df

In [22]:
#3.0.1  Bitcoin API - Daily Basis

In [23]:
# Import Function for Several Feature Variables from Glassnode
def import_btc_daily(feature_list):
    # Counter to Differentiate between Data Frame Setup and Data Frame Join
    counter=0
    
    # For Loop through all Features in the List
    for feature in feature_list:
        
        # URL
        url = 'https://api.glassnode.com/v1/metrics/{}?a=btc&f=json&api_key={}'.format(feature, API_Key)
        
        # Execute Import Function for a Single Feature Variable
        df=import_glassnode(url, feature)
        
        # Display Data Frame Tail
#         print(df.head())
#         print(df.tail())
        
        
        # Condition - Data Frame Set up or Data Frame Join
        if counter > 0:
            
            # Data Frame Join
            df_new = df_new.join(df, how='inner')   
        
        else:
            
            # Data Frame Setup
            df_new = df
        
        # Increase Counter
        counter +=1
    
    return df_new

In [24]:
#3.0.3  Ethereum API - Daily Basis

In [25]:
# Import Function for Several Feature Variables from Glassnode
def import_eth_daily(feature_list):
    # Counter to Differentiate between Data Frame Setup and Data Frame Join
    counter=0
    
    # For Loop through all Features in the List
    for feature in feature_list:
        # URL
        url = 'https://api.glassnode.com/v1/metrics/{}?a=eth&i=24h&f=json&api_key={}'.format(feature, API_Key)
        
        # Execute Import Function for a Single Feature Variable
        df=import_glassnode(url, feature)
        
        print(feature)
        
        # Display Data Frame Tail
        print(df.head())
        print(df.tail())
        # Condition - Data Frame Set up or Data Frame Join
        if counter > 0:
            # Data Frame Join
            df_new = df_new.join(df, how='inner')   
        else:
            # Data Frame Setup
            df_new = df
        
        # Increase Counter
        counter +=1
    
    return df_new

In [26]:
#3.1  Bitcoin Feature

In [27]:
#3.1.1  Bitcoin Feature Daily - Tier 1

In [28]:
btc_tier1_list_d = ['addresses/new_non_zero_count', 'addresses/active_count', 'addresses/sending_count', 'addresses/receiving_count', 'addresses/count', 
              'blockchain/block_height', 'blockchain/block_count', 'blockchain/block_interval_mean', 'blockchain/block_interval_median', 'blockchain/block_size_sum', 
              'blockchain/block_size_mean', 'fees/volume_sum', 'fees/volume_mean', 'fees/volume_median', 'indicators/sopr', 'market/price_usd_close', 'market/price_drawdown_relative', 'market/marketcap_usd', 
              'mining/difficulty_latest', 'mining/hash_rate_mean', 'supply/current', 'transactions/count', 'transactions/rate', 'transactions/size_sum', 
              'transactions/size_mean', 'transactions/transfers_volume_sum', 'transactions/transfers_volume_mean', 'transactions/transfers_volume_median', 
              'blockchain/utxo_created_count', 'blockchain/utxo_spent_count', 'blockchain/utxo_count', 'blockchain/utxo_created_value_sum', 'blockchain/utxo_created_value_mean',
              'blockchain/utxo_created_value_median', 'blockchain/utxo_spent_value_sum', 'blockchain/utxo_spent_value_mean', 'blockchain/utxo_spent_value_median']

In [29]:
#3.1.3  Bitcoin Feature Daily - Tier 2

In [30]:
btc_tier2_list_d = ['addresses/non_zero_count', 'addresses/min_point_zero_1_count', 'addresses/min_point_1_count', 'addresses/min_1_count', 'addresses/min_10_count',
                   'addresses/min_100_count', 'addresses/min_1k_count', 'addresses/min_10k_count', 'transactions/transfers_volume_to_exchanges_sum', 
                    'transactions/transfers_volume_from_exchanges_sum', 'transactions/transfers_volume_exchanges_net', 'transactions/transfers_to_exchanges_count',
                    'transactions/transfers_from_exchanges_count', 'transactions/transfers_volume_to_exchanges_mean', 'transactions/transfers_volume_from_exchanges_mean',
                    'distribution/balance_exchanges', 'fees/fee_ratio_multiple', 'indicators/sopr_adjusted', 
                    'indicators/net_unrealized_profit_loss', 'indicators/unrealized_profit', 'indicators/unrealized_loss', 'indicators/realized_profit',
                    'indicators/realized_loss', 'indicators/net_realized_profit_loss', 'indicators/puell_multiple', 'indicators/cdd',
                    'indicators/cdd_supply_adjusted', 'indicators/cdd_supply_adjusted_binary', 'indicators/reserve_risk', 'indicators/liveliness', 
                    'indicators/average_dormancy', 'indicators/average_dormancy_supply_adjusted',  'indicators/asol', 'indicators/msol', 
                    'indicators/sol_1h', 'indicators/sol_1h_24h', 'indicators/sol_1d_1w', 'indicators/sol_1w_1m', 'indicators/sol_1m_3m', 
                    'indicators/sol_3m_6m', 'indicators/sol_6m_12m', 'indicators/sol_1y_2y', 'indicators/sol_2y_3y', 'indicators/sol_3y_5y', 'indicators/sol_5y_7y',
                    'indicators/sol_7y_10y', 'indicators/sol_more_10y', 'indicators/nvt', 'indicators/nvts', 'indicators/velocity', 'indicators/stock_to_flow_deflection',
                    'indicators/difficulty_ribbon_compression', 'indicators/cvdd', 'indicators/rhodl_ratio', 'market/price_realized_usd',
                    'market/marketcap_realized_usd', 'market/mvrv', 'market/mvrv_z_score', 'mining/thermocap', 'mining/marketcap_thermocap_ratio', 'mining/revenue_sum',
                    'mining/revenue_from_fees', 'mining/volume_mined_sum', 'supply/profit_relative', 'supply/profit_sum', 'supply/loss_sum',
                    'supply/active_more_1y_percent', 'supply/active_more_2y_percent', 'supply/active_more_3y_percent', 'supply/active_more_5y_percent',
                    'supply/active_24h', 'supply/active_1d_1w', 'supply/active_1w_1m', 'supply/active_1m_3m', 'supply/active_3m_6m', 'supply/active_6m_12m', 'supply/active_1y_2y',
                    'supply/active_2y_3y', 'supply/active_3y_5y', 'supply/active_5y_7y', 'supply/active_7y_10y', 'supply/active_more_10y', 'supply/issued', 'supply/inflation_rate',
                    'supply/current_adjusted', 'transactions/transfers_volume_adjusted_sum', 'transactions/transfers_volume_adjusted_mean', 'transactions/transfers_volume_adjusted_median',
                    'blockchain/utxo_profit_relative', 'blockchain/utxo_profit_count', 'blockchain/utxo_loss_count']


In [31]:
#3.2  Bitcoin Data

In [32]:
#3.2.1  Bitcoin Data Daily Tier 1

In [33]:
# Set Working Directory to Tier 1 BTC Daily
os.chdir('{}/Tier1/BTC_Daily'.format(setdir))

In [34]:
df_btc_tier1_d = import_btc_daily(btc_tier1_list_d)

In [35]:
df_btc_tier1_d.tail()

,addresses/new_non_zero_count,addresses/active_count,addresses/sending_count,addresses/receiving_count,addresses/count,blockchain/block_height,blockchain/block_count,blockchain/block_interval_mean,blockchain/block_interval_median,blockchain/block_size_sum,...,transactions/transfers_volume_median,blockchain/utxo_created_count,blockchain/utxo_spent_count,blockchain/utxo_count,blockchain/utxo_created_value_sum,blockchain/utxo_created_value_mean,blockchain/utxo_created_value_median,blockchain/utxo_spent_value_sum,blockchain/utxo_spent_value_mean,blockchain/utxo_spent_value_median
Date,,,,,,,,,,,,,,,,,,,,,
2021-02-15,535064.00,1142365.00,708689.00,778833.00,785044737.00,670789.00,152.00,589.99,359.00,201027196.00,...,0.02,974929.00,946160.00,118789246.00,2374770.78,2.44,0.00,2373820.78,2.51,0.00
2021-02-16,510432.00,991949.00,552389.00,745448.00,785555169.00,670910.00,121.00,716.81,536.00,156916096.00,...,0.02,932767.00,715012.00,119007001.00,2515309.89,2.70,0.00,2514553.64,3.52,0.01
2021-02-17,555803.00,1138578.00,630336.00,829211.00,786110972.00,671048.00,138.00,620.64,395.00,178160687.00,...,0.02,1027930.00,823593.00,119211338.00,2077696.30,2.02,0.00,2076833.80,2.52,0.01
2021-02-18,560991.00,1138749.00,637433.00,820184.00,786671963.00,671185.00,137.00,630.19,521.00,178791889.00,...,0.02,1020323.00,819863.00,119411798.00,2202921.05,2.16,0.00,2202064.80,2.69,0.01
2021-02-19,603519.00,1246492.00,712634.00,884297.00,787275482.00,671338.00,153.00,568.91,375.00,203282199.00,...,0.02,1108810.00,922508.00,119598100.00,2365992.80,2.13,0.00,2365036.55,2.56,0.01


In [36]:
df_btc_tier1_d.shape

(3871, 37)

In [37]:
# 3.2.3  Bitcoin Data Daily Tier 2

In [38]:
# Set Working Directory to Tier 2 BTC Daily
os.chdir('{}/Tier2/BTC_Daily'.format(setdir))

In [ ]:
df_btc_tier2_d = import_btc_daily(btc_tier2_list_d)

In [ ]:
df_btc_tier2_d.tail()

In [ ]:
for i, word in enumerate(btc_tier1_list_d):
    btc_tier1_list_d[i] = btc_tier1_list_d[i].replace('/', '_')
    
for i, word in enumerate(btc_tier2_list_d):
    btc_tier2_list_d[i] = btc_tier2_list_d[i].replace('/', '_')


In [ ]:
#3.3  Ethereum Feature

In [ ]:
#3.3.1  Ethereum Feature Daily - Tier 1

In [ ]:
eth_tier1_list_d = ['addresses/new_non_zero_count', 'addresses/active_count', 'addresses/sending_count', 'addresses/receiving_count', 'addresses/count', 
              'blockchain/block_height', 'blockchain/block_count', 'blockchain/block_interval_mean', 'blockchain/block_interval_median', 'blockchain/block_size_sum', 
              'blockchain/block_size_mean', 'fees/volume_sum', 'fees/volume_mean', 'fees/volume_median', 'fees/gas_used_sum', 'fees/gas_used_mean', 'fees/gas_used_median',
            'fees/gas_price_mean', 'fees/gas_price_median', 'fees/gas_limit_tx_mean', 'fees/gas_limit_tx_median', 'indicators/sopr', 'market/price_usd_close', 'market/price_drawdown_relative', 'market/marketcap_usd', 
              'mining/difficulty_latest', 'mining/hash_rate_mean',  
            'supply/current', 'transactions/count', 'transactions/rate', 'transactions/transfers_count', 'transactions/transfers_rate', 'transactions/transfers_volume_sum', 
                  'transactions/transfers_volume_mean', 'transactions/transfers_volume_median'
                   ]

# 'protocols/uniswap_volume_sum', 'protocols/uniswap_transaction_count',   

In [ ]:
# 3.3.3  Ethereum Feature Daily - Tier 2

In [ ]:
eth_tier2_list_d = ['addresses/non_zero_count', 'addresses/min_point_zero_1_count', 'addresses/min_point_1_count', 'addresses/min_1_count', 'addresses/min_10_count',
                   'addresses/min_100_count', 'addresses/min_1k_count', 'addresses/min_10k_count', 'addresses/min_32_count', 'distribution/balance_1pct_holders',
                    'distribution/gini', 'distribution/herfindahl', 'distribution/supply_contracts', 
                    'transactions/transfers_volume_to_exchanges_sum', 'transactions/transfers_volume_from_exchanges_sum', 'transactions/transfers_volume_exchanges_net', 
                    'transactions/transfers_to_exchanges_count','transactions/transfers_from_exchanges_count', 'transactions/transfers_volume_to_exchanges_mean', 
                    'transactions/transfers_volume_from_exchanges_mean','distribution/balance_exchanges', 'fees/fee_ratio_multiple', 
                'indicators/net_unrealized_profit_loss', 'indicators/unrealized_profit', 'indicators/unrealized_loss', 'indicators/cdd', 'indicators/liveliness', 
                    'indicators/average_dormancy', 'indicators/asol', 'indicators/msol', 'indicators/nvt', 'indicators/nvts', 'indicators/velocity', 
                    'market/marketcap_realized_usd', 'market/mvrv', 'mining/thermocap', 'mining/marketcap_thermocap_ratio', 'mining/revenue_sum', 'mining/revenue_from_fees', 
                    'supply/profit_relative', 'supply/profit_sum', 'supply/loss_sum',
                    
                    
                    'supply/active_24h', 'supply/active_1d_1w', 'supply/active_1w_1m', 'supply/active_1m_3m', 'supply/active_3m_6m', 'supply/active_6m_12m', 'supply/active_1y_2y',
                    'supply/active_2y_3y', 'supply/active_3y_5y', 'supply/active_5y_7y', 'supply/active_7y_10y', 'supply/active_more_10y', 'supply/issued', 'supply/inflation_rate',
                    'transactions/contract_calls_internal_count']  

In [ ]:
#3.4  Ethereum Data

In [ ]:
#3.4.1  Ethereum Data Daily - Tier 1

In [ ]:
df_eth_tier1_d = import_eth_daily(eth_tier1_list_d)

In [ ]:
df_eth_tier1_d.tail()

In [ ]:
df_eth_tier1_d.shape

In [ ]:
#3.5.1  Ethereum Data Daily - Tier 2

In [ ]:
# Set Working Directory to Tier 2 ETH Daily
os.chdir('{}/Tier2/ETH_Daily'.format(setdir))

In [ ]:
df_eth_tier2_d = import_eth_daily(eth_tier2_list_d)

In [ ]:
df_eth_tier2_d

In [ ]:
for i, word in enumerate(eth_tier1_list_d):
    eth_tier1_list_d[i] = eth_tier1_list_d[i].replace('/', '_')

for i, word in enumerate(eth_tier2_list_d):
    eth_tier2_list_d[i] = eth_tier2_list_d[i].replace('/', '_')


In [ ]:
df_dataset2_eth = pd.merge(df_eth_tier1_d, df_eth_tier2_d, how='inner', left_on='Date', right_on='Date')

In [ ]:
df_dataset2_btc = pd.merge(df_btc_tier1_d, df_btc_tier2_d, how='inner', left_on='Date', right_on='Date')

In [ ]:
#3.6  Target Variables

In [ ]:
df_dataset2_btc['Closed Price USD'] = df_dataset2_btc['market/price_usd_close']
df_dataset2_eth['Closed Price USD'] = df_dataset2_eth['market/price_usd_close']

In [ ]:
try:
    df_dataset2_btc = df_dataset2_btc.drop('market/price_usd_close', axis = 1)
except:
    print("was already gone")
    
try:
    df_dataset2_eth = df_dataset2_eth.drop('market/price_usd_close', axis = 1)
except:
    print("was already gone")   


In [ ]:
df_dataset2_btc['Daily Return in USD'] = df_dataset2_btc['Closed Price USD'].diff()
df_dataset2_eth['Daily Return in USD'] = df_dataset2_eth['Closed Price USD'].diff()

In [ ]:
# Add Column with BTC Daily Returns in Percent
df_dataset2_btc['Daily Return in Percent'] = (df_dataset2_btc['Daily Return in USD']) / (df_dataset2_btc['Closed Price USD'].shift(1))*100
# Add Column with BTC Daily Returns in Percent
df_dataset2_eth['Daily Return in Percent'] = (df_dataset2_eth['Daily Return in USD']) / (df_dataset2_eth['Closed Price USD'].shift(1))*100

In [ ]:
# Add Column with BTC Daily Log Returns in Percent
df_dataset2_btc['Daily Log Return in Percent'] = pd.DataFrame(np.log(df_dataset2_btc['Closed Price USD']/df_dataset2_btc['Closed Price USD'].shift(1))*100)
# Add Column with BTC Daily Log Returns in Percent
df_dataset2_eth['Daily Log Return in Percent'] = pd.DataFrame(np.log(df_dataset2_eth['Closed Price USD']/df_dataset2_eth['Closed Price USD'].shift(1))*100)

In [ ]:
df_dataset2_btc['Log Price in USD'] = np.log(df_dataset2_btc['Closed Price USD'])
df_dataset2_eth['Log Price in USD'] = np.log(df_dataset2_eth['Closed Price USD'])

In [ ]:
df_dataset2_btc = df_dataset2_btc.dropna()

In [ ]:
df_dataset2_eth = df_dataset2_eth.dropna()

In [ ]:
os.chdir('{}'.format(setdir))

In [ ]:
# Creating CSV-File of Blockchain and Economic Data
df_dataset2_btc.to_csv('df_btc_tier1_tier2.csv')

In [ ]:
# Creating CSV-File of Blockchain and Economic Data
df_dataset2_eth.to_csv('df_eth_tier1_tier2.csv')

In [ ]:
# Current Working Directory
os.getcwd()